In [57]:
import dash
from dash import dcc, html, Input, Output, State
import dash.dash_table
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import sample_colorscale

In [58]:
# Load the merged data
df = pd.read_csv("../results/dash_expression_data.csv", na_values=["NA", "NaN", ""])

# Clean for safety
df['gene'] = df['gene'].astype(str)
df['condition'] = df['condition'].astype(str)
# Convert to numeric, ensuring float64 dtype
df["log2FoldChange"] = pd.to_numeric(df["log2FoldChange"], errors="coerce", downcast=None)
df['FDR'] = pd.to_numeric(df['FDR'], errors='coerce', downcast=None)
#df["FDR"] = pd.to_numeric(df["FDR"], errors="coerce")

# Set up gene list
genes = sorted(df['gene'].unique())

In [59]:
print(df)
#df['FDR'].min(skipna=True)
#df['FDR'].max(skipna=True)
#print(df.columns.tolist())
#print(df["FDR"].head(10))
#print(df["FDR"].dtype)
#range(df['FDR'])

                   gene sample  expression     ID condition          Run  \
0                PRDM16   TLE1    3.321570   TLE1       TLE  SRR10446605   
1                PRDM16   TLE2    3.558254   TLE2       TLE  SRR10446608   
2                PRDM16   TLE3    3.117902   TLE3       TLE  SRR10446611   
3                PRDM16  Ctrl1    3.471156  Ctrl1        TL  SRR10446596   
4                PRDM16  Ctrl2    3.739109  Ctrl2        TL  SRR10446599   
...                 ...    ...         ...    ...       ...          ...   
159337  ENSG00000300815   TLE2    3.434051   TLE2       TLE  SRR10446608   
159338  ENSG00000300815   TLE3    2.563037   TLE3       TLE  SRR10446611   
159339  ENSG00000300815  Ctrl1    3.650406  Ctrl1        TL  SRR10446596   
159340  ENSG00000300815  Ctrl2    2.987791  Ctrl2        TL  SRR10446599   
159341  ENSG00000300815  Ctrl3    2.973240  Ctrl3        TL  SRR10446602   

        sizeFactor  log2FoldChange  FDR  
0         0.681726             NaN  NaN  
1  

In [103]:
# Dash app setup
app = dash.Dash(__name__)

default_gene = df.loc[df['log2FoldChange'].abs().idxmax(), 'gene']

#Viridis color palette
viridis_colors = sample_colorscale("Viridis", [0.75, 0.25])

app.layout = html.Div([
  html.H2("Gene Expression Dashboard", style={"textAlign": "center"}),
  dcc.Checklist(
    id="volcano-toggle",
    options=[{"label": " Show volcano plot", "value": "show"}],
    value=[],
    style={"margin": "10px 0"}
),
  html.Div(id="volcano-container"),

  dcc.Dropdown(
    id="gene-dropdown",
    options=[{"label": g, "value": g} for g in sorted(df["gene"].dropna().unique())],
    # Set a specific gene of interest as the default gene
    #value="LGR6",
    value=default_gene,
    style={"width": "200px"}
  ),
  dcc.Graph(id="expression-plot",
           style={"width": "100%",
                  "margin": "0 auto"}
           ),
  dcc.Checklist(
    id='sig-only-toggle',
    options=[{'label': ' Show only significant genes (FDR < 0.05)', 'value': 'sig'}],
    value=[],  # empty = show all by default
    style={"margin": "10px"}
),
  html.Button("Download graph data (CSV)", id="download-btn", n_clicks=0, style={"margin": "20px"}),
  dcc.Download(id="download-data"),
  html.Div(id="de-info-text", style={"margin": "10px", "textAlign": "center", "fontSize": "14px"}),
  dash.dash_table.DataTable(
    id='expression-table',
    style_table={'overflowX': 'auto', 'margin': '20px auto', 'width': '70%'},
    style_cell={'textAlign': 'left',
                'padding': '5px',
                'fontFamily': 'sans-serif',
                'fontSize': '14px'
               },
    style_data_conditional=[
        {
            'if': {'column_id': 'expression'},
            'textAlign': 'right',
            'fontFamily': 'monospace'
        }
    ],
    style_header={'fontWeight': 'bold'},
    page_size=10
  )
])

@app.callback(
    Output("volcano-container", "children"),
    Input("volcano-toggle", "value")
)
def show_volcano(toggle_vals):
    if "show" not in toggle_vals:
        return None

    volcano_df = df.copy()
    volcano_df["-log10FDR"] = -np.log10(volcano_df["FDR"])
    volcano_df["Significant"] = volcano_df["FDR"] < 0.05

    fig = px.scatter(
        volcano_df,
        x="log2FoldChange",
        y="-log10FDR",
        color="Significant",
        hover_name="gene",
        custom_data=["gene"],  # needed for click-to-select
        color_discrete_map={True: "crimson", False: "gray"}
    )

    fig.update_traces(marker=dict(size=6, opacity=0.7))
    fig.update_layout(
        title="Volcano Plot",
        height=450,
        margin=dict(t=50, l=40, r=20, b=40),
        xaxis_title="log₂ Fold Change",
        yaxis_title="-log₁₀(FDR)",
        plot_bgcolor="white"
    )
    fig.update_xaxes(linecolor="black", zeroline=True, zerolinewidth=1)
    fig.update_yaxes(linecolor="black")

    return dcc.Graph(id="volcano-plot", figure=fig)
                                                                    
@app.callback(
  Output("gene-dropdown", "options"),
  Output("gene-dropdown", "value"),
  Input("sig-only-toggle", "value")
)

def update_gene_list(toggle_values):
    if "sig" in toggle_values:
        filtered = df[df["FDR"] < 0.05]
    else:
        filtered = df

    # Remove NAs and duplicates
    gene_list = sorted(filtered["gene"].dropna().unique())

    # Default gene: max abs log2FC
    if len(gene_list) == 0:
        return [], None
    default_gene = (
        filtered.loc[filtered["log2FoldChange"].abs().idxmax(), "gene"]
        if "log2FoldChange" in filtered.columns else gene_list[0]
    )

    options = [{"label": g, "value": g} for g in gene_list]
    
    return options, default_gene


@app.callback(
  Output("expression-plot", "figure"),
  Input("gene-dropdown", "value")
)

def update_plot(selected_gene):
    #Define dataset for the plot
    data = df[df["gene"] == selected_gene].copy()

    #Define FDR and LFC text
    padj = data["FDR"].iloc[0]
    lfc = data["log2FoldChange"].iloc[0]
    padj_text = f"FDR = {padj:.2e}" if pd.notnull(padj) else "No DE result"

    fig = go.Figure()

    #Convert conditions from catagorical to numeric to plot boxplots and jittered datapoints on the same field
    conditions = ["TL", "TLE"]  # TL = Control, TLE = Epilepsy
    cond_to_x = {cond: i for i, cond in enumerate(conditions)}
    #Add condition labels.
    cond_labels = {
    "TL": "Control",
    "TLE": "Epilepsy"
    }
    
    for i, cond in enumerate(conditions):
        subset = data[data["condition"] == cond]
        x_val = cond_to_x[cond]
        
        #Boxplot
        fig.add_trace(go.Box(
            y=subset["expression"],
            x=[x_val] * len(subset),
            name=cond,
            marker_color="white", #fill color
            fillcolor="white", # explicitly set
            line=dict(color="black"), #line color
            width=0.25,
            boxpoints="outliers",
            showlegend=False
        ))

        # Dot plot (viridis-colored with jitter)
        x_jittered = [x_val + np.random.uniform(-0.15, 0.15) for _ in range(len(subset))]

        #Scatter points
        fig.add_trace(go.Scatter(
            y=subset["expression"],
            x=x_jittered,
            mode="markers",
            marker=dict(
                size=12, 
                color=viridis_colors[i % len(viridis_colors)]
            ),
            hovertext=subset["sample"],
            showlegend=False
        ))

        #Add significance bar for DE genes
        # Check if gene is significant
        fdr = data["FDR"].iloc[0]
        lfc = data["log2FoldChange"].iloc[0]
        is_significant = pd.notnull(fdr) and fdr < 0.05
        #y_max = data["expression"].max()
        #bar_y = y_max * 1.2  # pad above the data

        if is_significant:
            y_max = data["expression"].max()
            bar_y = y_max * 1.05
        
            # Horizontal line
            fig.add_trace(go.Scatter(
                x=[0, 1],
                y=[bar_y, bar_y],
                mode="lines",
                line=dict(color="black", width=1),
                showlegend=False,
                hoverinfo="skip"
            ))
        
            # Vertical line 0
            fig.add_trace(go.Scatter(
                x=[0, 0],
                y=[(bar_y*0.98), bar_y],
                mode="lines",
                line=dict(color="black", width=1),
                showlegend=False,
                hoverinfo="skip"
            ))

            # Vertical line 1
            fig.add_trace(go.Scatter(
                x=[1, 1],
                y=[(bar_y*0.98), bar_y],
                mode="lines",
                line=dict(color="black", width=1),
                showlegend=False,
                hoverinfo="skip"
            ))

    fig.update_layout(
        title=f"<i>{selected_gene}</i>",
        yaxis_title="Expression (log scale)",
        title_x=0.5,
        font=dict(size=12),
        plot_bgcolor="white",
        xaxis=dict(
        range=[-0.5, 1.5],  # tighter than default for 2 categories
        ),
        annotations=[
            dict(
                text=padj_text,
                x=0.5,
                y=1.15,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=12),
            )
        ]
    )

    fig.update_yaxes(type="log", linecolor="black")
    fig.update_xaxes(
        linecolor="black",
        tickangle=-45,
        tickmode="array",
        tickvals=list(cond_to_x.values()),
        ticktext=[cond_labels[cond] for cond in conditions],
        type="linear"
    )

    return fig

@app.callback(
    Output("download-data", "data"),
    Input("download-btn", "n_clicks"),
    State("gene-dropdown", "value"),
    prevent_initial_call=True
)
def download_table(n_clicks, selected_gene):
    if selected_gene is None:
        return dash.no_update

    data_subset = df[df["gene"] == selected_gene].copy()

    columns_to_export = ["sample", "condition", "expression"]
    if "FDR" in data_subset.columns and "log2FoldChange" in data_subset.columns:
        columns_to_export += ["log2FoldChange", "FDR"]

    return dcc.send_data_frame(
        data_subset[columns_to_export].to_csv,
        filename=f"{selected_gene}_expression_table.csv",
        index=False
    )

@app.callback(
    Output("de-info-text", "children"),
    Input("gene-dropdown", "value")
)
def update_de_info(selected_gene):
    if selected_gene is None:
        return ""

    gene_data = df[df["gene"] == selected_gene]

    if "FDR" in gene_data.columns and "log2FoldChange" in gene_data.columns:
        fdr_val = gene_data["FDR"].iloc[0]
        lfc_val = gene_data["log2FoldChange"].iloc[0]

        if pd.notnull(fdr_val) and pd.notnull(lfc_val):
            return f"log₂FC: {lfc_val:.3f} | FDR: {fdr_val:.2e}"
        else:
            return "No differential expression data available for this gene."
    else:
        return "DE information not available."

@app.callback(
    Output("expression-table", "data"),
    Output("expression-table", "columns"),
    Input("gene-dropdown", "value")
)
def update_table(selected_gene):
    if selected_gene is None:
        return [], []

    data_subset = df[df["gene"] == selected_gene].copy()
    #Round expression data to 3 significant digits
    data_subset["expression"] = data_subset["expression"].apply(lambda x: f"{x:.4g}")

    #Choose what to show in the table
    columns_to_show = ["sample", "condition", "expression"]
    #Optionally include FDR, log2FC, or any metadata
    #if "FDR" in data_subset.columns and "log2FoldChange" in data_subset.columns:
    #    data_subset["FDR"] = data_subset["FDR"].round(4)
    #    data_subset["log2FoldChange"] = data_subset["log2FoldChange"].round(3)
    #    columns_to_show += ["log2FoldChange", "FDR"]
    #Replace columns for readability
    column_headers = {
    "sample": "Sample",
    "condition": "Condition",
    "expression": "Expression (log scale)"
    #"log2FoldChange": "log₂ Fold Change",
    #"FDR": "FDR (q-value)"
    }
    
    table_data = data_subset[columns_to_show].to_dict("records")
    table_columns = [{"name": column_headers[col], "id": col} for col in columns_to_show]

    return table_data, table_columns


if __name__ == "__main__":
  app.run(debug=True)

In [67]:
viridis_colors = sample_colorscale("Viridis", [0.75, 0.25])
print(viridis_colors)

['rgb(96, 200, 96)', 'rgb(59, 81, 138)']


In [68]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i + 2], 16) for i in (0, 2, 4))

# Example usage:
hex_color = "#3B528B"
rgb_color = hex_to_rgb(hex_color)
print(rgb_color)  # Output: (255, 0, 0)

(59, 82, 139)
